In [57]:
import pandas as pd
import numpy as np
from sklearn import metrics
import keras
import pickle
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from vecstack import stacking

In [58]:
df = pd.read_csv('ilk_asama_data.csv')
data = df.copy()
print(data.shape)
data.head()

(14882, 2)


,line,y
0,şarkıcı norma zimmer ile iyi bir ikili oluşturdu.,1
1,gemiler kariyerleri boyunca sık sık birlikte h...,1
2,ayrıca; yarı x iletken özellikleri vardır.,0
3,futbol oynadığı dönemlerde orta saha pozisyonu...,1
4,i̇ki fenotip ayrım yapmadan bir arada ürerler.,1


In [59]:
X = data['line']
y = data['y']

In [60]:
vocab_size = 15000
tokenizer = Tokenizer(vocab_size)
tokenizer.fit_on_texts(X)
list_tokenized_train = tokenizer.texts_to_sequences(X)

In [61]:
maxlen = 7
X_train_padded = pad_sequences(list_tokenized_train, maxlen=maxlen)

In [62]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_train_padded, y, test_size=0.2, stratify=y)

In [63]:
y_train.value_counts()

1    5953
0    5952
Name: y, dtype: int64

In [64]:
y_test.value_counts()

0    1489
1    1488
Name: y, dtype: int64

# Birinci Seviye Algoritmalar

In [65]:
from xgboost import XGBClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

In [66]:
models = [
    ExtraTreesClassifier(random_state=0,n_estimators=100, max_depth=17),
        
    LogisticRegression(solver ='lbfgs',max_iter= 1000,random_state=0, multi_class='ovr',class_weight='balanced'),
    
    SVC(gamma='auto',random_state=0)
    
]

## Stacking

In [67]:
S_train, S_test = stacking(models,X_train, y_train, X_test,regression=False,mode='oof_pred',needs_proba=False,save_dir=None,n_folds=10,stratified=True,shuffle=True,verbose=2) 

task:         [classification]
n_classes:    [2]
metric:       [accuracy_score]
mode:         [oof_pred]
n_models:     [3]

model  0:     [ExtraTreesClassifier]
    fold  0:  [0.75986566]
    fold  1:  [0.73131822]
    fold  2:  [0.74223342]
    fold  3:  [0.72795970]
    fold  4:  [0.72292191]
    fold  5:  [0.74117647]
    fold  6:  [0.71764706]
    fold  7:  [0.73613445]
    fold  8:  [0.74117647]
    fold  9:  [0.74705882]
    ----
    MEAN:     [0.73674922] + [0.01171815]
    FULL:     [0.73674927]

    Fitting on full train set...

model  1:     [LogisticRegression]
    fold  0:  [0.55667506]
    fold  1:  [0.57430730]
    fold  2:  [0.56926952]
    fold  3:  [0.55919395]
    fold  4:  [0.55331654]
    fold  5:  [0.54285714]
    fold  6:  [0.55546218]
    fold  7:  [0.56134454]
    fold  8:  [0.55882353]
    fold  9:  [0.56134454]
    ----
    MEAN:     [0.55925943] + [0.00813007]
    FULL:     [0.55926081]

    Fitting on full train set...

model  2:     [SVC]
    fold  0:  [0.4

# İkinci Seviye Model XGBoost

In [68]:
model = XGBClassifier(learning_rate=0.5,eval_metric='logloss',
                       use_label_encoder=False,n_estimators=100,max_depth=7,random_state=0)
    
# Fit 2nd level model
model = model.fit(S_train, y_train)

# Predict
y_pred = model.predict(S_test)


In [69]:
from sklearn.metrics import classification_report, confusion_matrix
print(confusion_matrix(y_test,y_pred))
print(classification_report(y_test,y_pred))

[[1098  391]
 [ 406 1082]]
              precision    recall  f1-score   support

           0       0.73      0.74      0.73      1489
           1       0.73      0.73      0.73      1488

    accuracy                           0.73      2977
   macro avg       0.73      0.73      0.73      2977
weighted avg       0.73      0.73      0.73      2977

